# Exercise 4

Coded by:
1. Carmen Eitel - carmen.eitel@student.kit.edu

on 01.05.2019

version 1.0

## Bibliotheken und Einstellungen

In [9]:
# import libraries 
import numpy as np
import pandas as pd
import sklearn

## Teilaufgaben

### a) Daten einlesen und erkunden
Lesen Sie den Datensatz Credit.csv ein. Erkunden Sie die Daten und konvertieren Sie gegebenenfalls kategorische Variablen in ein geeignetes Format (z.B. mit pandas.get_dummies() oder sklearn.model_selection.OneHotEncoder()). 

In [20]:
# load dataset
data = pd.read_csv('./data/Credit.csv')

# first insight of data by checking the first 5 data samples
data.head()

,Unnamed: 0,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [53]:
# extract features

# target variable, integer
balance = np.array(data["Balance"])

# integer
limit = np.array(data["Limit"])
rating = np.array(data["Rating"])
cards = np.array(data["Cards"])
age = np.array(data["Age"])
education = np.array(data["Education"])

# float
income = np.array(data["Income"])


# binary, converted to dummy dataframe
gender_df = pd.get_dummies(data["Gender"]) # np.array(pd.get_dummies(data["Gender"]))
student_df = pd.get_dummies(data["Student"]) # np.array(pd.get_dummies(data["Student"]))
married_df = pd.get_dummies(data["Married"])

# categorical
ethnicity_raw = pd.Series(data["Ethnicity"], dtype="category") 
ethnicity_raw.head()


0    Caucasian
1        Asian
2        Asian
3        Asian
4    Caucasian
Name: Ethnicity, dtype: category
Categories (3, object): [African American, Asian, Caucasian]

In [59]:
# there are three values for ethnicities: African American, Asian, Caucasian
# convert them into three dummies

african_american_df = pd.get_dummies(ethnicity_raw == "African American")
asian_df = pd.get_dummies(ethnicity_raw == "Asian")
caucasian_df = pd.get_dummies(ethnicity_raw == "Caucasian")

TODO: weiter erkunden, visualisieren etc...
TODO: vermutlich als dataFrame zusammen geschickter

### b) Datensatz teilen
Separieren Sie den Datensatz in 80%-20% für Training und Testdatensatz. Was müssen Sie hier beachten?

TODO: shuffle, split pandas.DataFrame.sample

### c) OLS mit allen Variablen
Erstellen Sie aus der Variable Ethnicity drei binäre Variablen Ethnicity_Caucasian, Ethnicity_Asian und Ethnicity_African. Führen Sie eine lineare Regression mit OLS als Schätzer mit diesen Variablen und allen anderen Variablen (außer Limit) durch. Berechnen Sie Root Mean Square Error (RMSE) zwischen der Vorhersage und den wahren Werteni m Test-Datensatz. Beschreiben Sie Ihre Ergebnisse.

aus a kopieren

### d) OLS ohne Variable Ethnicity_Caucasian
Führen Sie die Analyse in 4c) erneut durch, ohne die Variable Ethnicity_Caucasian. Vergleichen Sie diese Ergebnisse mit denen aus 4c).

### e) OLS mit unterschiedlichen Variablen
Führen Sie eine lineare Regression (OLS) mit folgenden Variablendurch: 
1. Income, Limit, Age
2. Income, Rating, Age
3. Income, Limit, Rating, Age 

Beschreiben Sie die Koeffizienten, Konfidenzintervalle und Signifikanzniveau. Vergleichen Sie die Ergebnisse.

### f) Korrelation
Was ist Korrelation? Wann benutzt man Pearson Korrelation, Spearman Korrelation und Kendalls Tau? Berechnen Sie Korrelationen zwischen allen Variablen im Trainings-Datensatz. Welche Schlussfolgerung können Sie daraus ziehen?

Anmerkung: $*$: Multiplikationszeichen
#### Was ist Korrelation?
Korrelation misst die Stärke und Richtung des Zusammenhangs bzw. der Abhängigkeit zwischen zwei Zufallsvariablen. (siehe Skript Statistik XXX Seite 58)
Die Korrelation ist nicht mit Kausalität gleichzusetzen, da zwar ein Zusammenhang zwischen zwei Zufallsvariablen möglich ist, ohne dass diese sich direkt beeinflussen.

#### Pearson Korrelation
Das bekannteste skalare Zusammenhangsmaß zweier Zufallsvariablen X und Y ist der Korrelationskoeffizient
nach Bravais-Pearson:

$$Corr(X,Y) = \rho_{BP} = \dfrac{Cov(X,Y)}{\sqrt{X}*\sqrt{Y}}$$

Er misst die Stärke des linearen Zusammenhangs von X und Y und hängt von den
Randverteilungen von X und Y ab. siehe skript seite 58, 59
Sein Wertebereich ist $[-1,1]$.

#### Spearman Korrelation
Das Korrelationsmaß Spearmans $\rho_S$ ist unabhängig von der Randverteilung und somit ein Rangkorrelationskoeffizient.
$$\rho_S = \dfrac{Cov(F_X(X), F_Y(Y))}{\sqrt{Var(F_X(X))}*\sqrt{F_Y(Y)}} = \dfrac{Cov(U, V)}{\sqrt{Var(U)}*\sqrt{Var(V)}}$$

$F_Z(Z) ist die Verteilungsfunktion der Zufallsvariable $Z$

Als Rangkorrelationskoeffizient ist Spearmans's $\rho_S$ invariant gegenüber streng monoton wachsende Transformationen $\alpha(.)$ und $\beta(.)$ der Ränder: 

$$ \rho_{S,X,Y} = \rho_{S,\alpha(x), \beta(Y)} $$

Auch hier ist der Wertebereich $[-1,1]$, jedoch wird nicht die Stärke des linearen Zusammenhangs zwischen den Zufallsvariablen X und Y, sondern zwischen U und V und somit den monotonen Zusammenhangs zwischen X und Y.


##### Kendalls $\tau$


#### Wann verwendet man was

#### Berechnungen und Schlussfolgerung

### g) Random Forest
Führen Sie die Analyse in 4e) mit Random Forestanstatt der linearen Regression durch. Variieren Sie dabei den Parameter random_statemit den Werten {1, 33, 135, 123, 99, 22} und einigen selbst gewählten Werten. Berechnen Sie MSE des Test-Datensatzes. Vergleichen Sie Feature Importance. Welche Schlussfolgerung können Sie daraus ziehen?

### h) Multikollinearität
Was ist Multikollinearität? Was sind ihre möglichen Ursachen und Auswirkungen? Wie kann man sie erkennen? Welche Maßnahmen gibt es, wenn Variablen in den Daten kollinear sind?